<a href="https://colab.research.google.com/github/tariqshaban/arabic-sign-language-image-classification/blob/master/Arabic%20Sign%20Language%20Image%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Dependencies

In [ ]:
# Display generic output messages
!pip install colorama

# Properly display Arabic characters in plots
!pip install arabic_reshaper
!pip install python-bidi

# Download assets from the GitHub repository
!apt install subversion
!svn checkout https://github.com/tariqshaban/arabic-sign-language-image-classification/trunk/assets

import arabic_reshaper
import os
import re
import random
import shutil
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from bidi.algorithm import get_display
from colorama import Fore, Back, Style
from enum import Enum
from keras import activations
from keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from keras.utils import plot_model
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from glob import glob
from IPython.display import Image, display, clear_output

# Helps to finalize the seed
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.random.set_seed(1)

clear_output()
print(Fore.GREEN + u'\u2713 ' + 'Successfully downloaded dependencies.')    
print(Style.RESET_ALL)

# Enumerations

### Enumerating Neural Network Types

In [ ]:
class NeuralNetworkType(Enum):
    FULLY_CONNECTED_NETWORK = 'FullyConnectedNetwork_INEFFECTIVE_SCRATCH'
    CONVOLUTIONAL_NEURAL_NETWORK = 'ConvolutionalNeuralNetwork_SCRATCH'
    CONVOLUTIONAL_NEURAL_NETWORK_WITH_TRANSFER_LEARNING = 'ConvolutionalNeuralNetworkWithTransferLearning'

### Enumerating Neural Network Sizes

In [ ]:
class NeuralNetworkSize(Enum):
    NANO = 'Nano_INEFFECTIVE'
    MICRO = 'Micro'
    SMALL = 'Small'

### Enumerating Hidden Layer Activation Function Types

In [ ]:
class HiddenLayerActivationFunctionType(Enum):
    SIGMOID = 'Sigmoid_INTENSIVE'
    TANH = 'Tanh_INTENSIVE'
    RELU = 'ReLU'
    LEAKY_RELU = 'LeakyReLU'

### Enumerating Optimizers

In [ ]:
class OptimizerType(Enum):
    SGD = 'SGD'
    RMS_PROP = 'RMSProp'
    ADAM = 'Adam'

# Global Variables

In [ ]:
CLASSES_DF = pd.read_excel('./assets/Labels/ClassLabels.xlsx', index_col='ClassId')
CLASSES_DF.sort_values('ClassAr', inplace=True)

CLASSES = CLASSES_DF['Class'].to_list()
CLASSES_AR = CLASSES_DF['ClassAr'].to_list()

TRAIN_SPLIT = 0.7
VALID_SPLIT = 0.2
TEST_SPLIT = 0.1

SOURCE_DIRECTORY = './assets/ArASL_Database_54K/'
REFACTORED_DIRECTORY = './assets/refactored_data/'
TRAIN_DIRECTORY = f'{REFACTORED_DIRECTORY}train/'
VALID_DIRECTORY = f'{REFACTORED_DIRECTORY}valid/'
TEST_DIRECTORY = f'{REFACTORED_DIRECTORY}test/'

BASE_MODEL = EfficientNetB0(weights='imagenet', include_top=False)
BASE_MODEL.trainable = False

PREPROCESSING_METHOD = tf.keras.applications.efficientnet.preprocess_input

# Helper Methods

### Get Neural Network

In [ ]:
def get_neural_network():
    if NEURAL_NETWORK_SIZE == NeuralNetworkSize.NANO:
        scalar = 2
    elif NEURAL_NETWORK_SIZE == NeuralNetworkSize.MICRO:
        scalar = 4
    elif NEURAL_NETWORK_SIZE == NeuralNetworkSize.SMALL:
        scalar = 6

    def _get_fully_connected_network_block(x):
        return layers.Dense(pow(2, scalar - x + 6), activation=get_hidden_layer_activation_function())

    def _get_convolutional_neural_network_block(x):
        return [
            layers.Conv2D(filters=pow(2, x + 4), kernel_size=3),
            layers.BatchNormalization(),
            layers.Activation(get_hidden_layer_activation_function()),
            layers.MaxPooling2D((2, 2))
        ]

    def _get_convolutional_neural_network_with_transfer_learning_block(x):
        return layers.Dense(pow(2, scalar - x + 4), activation=get_hidden_layer_activation_function())

    if NEURAL_NETWORK_TYPE == NeuralNetworkType.FULLY_CONNECTED_NETWORK:
        return \
            [layers.GlobalAveragePooling2D()] + \
            [_get_fully_connected_network_block(x) for x in range(1, scalar + 1)]
    elif NEURAL_NETWORK_TYPE == NeuralNetworkType.CONVOLUTIONAL_NEURAL_NETWORK:
        blocks = [_get_convolutional_neural_network_block(x) for x in range(1, scalar + 1)]
        return \
            [layer for block in blocks for layer in block] + \
            [
                layers.GlobalAveragePooling2D(),
                layers.Dense(64, activation=get_hidden_layer_activation_function()),
                layers.Dense(32, activation=get_hidden_layer_activation_function())
            ]
    elif NEURAL_NETWORK_TYPE == NeuralNetworkType.CONVOLUTIONAL_NEURAL_NETWORK_WITH_TRANSFER_LEARNING:
        return \
            [
                BASE_MODEL,
                layers.GlobalAveragePooling2D(),
                layers.Dropout(0.5),
            ] + \
            [_get_convolutional_neural_network_with_transfer_learning_block(x) for x in range(1, scalar)]

### Get Hidden Layer Activation Function

In [ ]:
def get_hidden_layer_activation_function():
    if HIDDEN_LAYER_ACTIVATION_FUNCTION_TYPE == HiddenLayerActivationFunctionType.SIGMOID:
        return activations.sigmoid
    elif HIDDEN_LAYER_ACTIVATION_FUNCTION_TYPE == HiddenLayerActivationFunctionType.TANH:
        return activations.tanh
    elif HIDDEN_LAYER_ACTIVATION_FUNCTION_TYPE == HiddenLayerActivationFunctionType.RELU:
        return activations.relu
    elif HIDDEN_LAYER_ACTIVATION_FUNCTION_TYPE == HiddenLayerActivationFunctionType.LEAKY_RELU:
        return activations.leaky_relu

### Get Optimizer

In [ ]:
def get_optimizer():
    if OPTIMIZER_TYPE == OptimizerType.SGD:
        return tf.keras.optimizers.SGD()
    elif OPTIMIZER_TYPE == OptimizerType.RMS_PROP:
        return tf.keras.optimizers.RMSprop()
    elif OPTIMIZER_TYPE == OptimizerType.ADAM:
        return tf.keras.optimizers.Adam()

### Finalize Seed

In [ ]:
def finalize_seed():
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)

### Prime Dataset

In [ ]:
def prime_dataset():
    if os.path.exists(REFACTORED_DIRECTORY):
        shutil.rmtree(REFACTORED_DIRECTORY)

    # Create Training, Validation, and Testing directories
    for label in CLASSES:
        os.makedirs(f'{TRAIN_DIRECTORY}{label}', exist_ok=True)
        os.makedirs(f'{VALID_DIRECTORY}{label}', exist_ok=True)
        os.makedirs(f'{TEST_DIRECTORY}{label}', exist_ok=True)

    # Partition Images into Training, Validation, and Testing
    for label in CLASSES:
        numOfFiles = len(next(os.walk(f'{SOURCE_DIRECTORY}{label}/'))[2])

        for image_file in random.sample(glob(f'{SOURCE_DIRECTORY}{label}/*'), int(numOfFiles * TRAIN_SPLIT)):
            shutil.move(image_file, f'{TRAIN_DIRECTORY}{label}')

        for image_file in random.sample(glob(f'{SOURCE_DIRECTORY}{label}/*'), int(numOfFiles * VALID_SPLIT)):
            shutil.move(image_file, f'{VALID_DIRECTORY}{label}')

        for image_file in glob(f'{SOURCE_DIRECTORY}{label}/*'):
            shutil.move(image_file, f'{TEST_DIRECTORY}{label}')

### Build Model

In [ ]:
def build_model(plot_network: bool = True, measure_performance: bool = True):
    train_batches = ImageDataGenerator(preprocessing_function=PREPROCESSING_METHOD).flow_from_directory(
        directory=TRAIN_DIRECTORY, classes=CLASSES, batch_size=128)
    valid_batches = ImageDataGenerator(preprocessing_function=PREPROCESSING_METHOD).flow_from_directory(
        directory=VALID_DIRECTORY, classes=CLASSES, batch_size=128, shuffle=False)
    test_batches = ImageDataGenerator(preprocessing_function=PREPROCESSING_METHOD).flow_from_directory(
        directory=TEST_DIRECTORY, classes=CLASSES, batch_size=128, shuffle=False)

    model = Sequential(
        get_neural_network() +
        [layers.Dense(len(CLASSES), activation='softmax')]
    )

    model.compile(optimizer=get_optimizer(), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=EARLY_STOPPING_PATIENCE)

    fitted_model = model.fit(x=train_batches, validation_data=valid_batches, epochs=EPOCHS, callbacks=[es])
    loss, accuracy = model.evaluate(x=test_batches, batch_size=128)

    print(Fore.GREEN + u'\n\u2713 ' + f'Loss ==> {loss}')
    print(Fore.GREEN + u'\n\u2713 ' + f'Accuracy ==> {accuracy}')

    if plot_network:
        plot_model(model, show_shapes=True, show_layer_activations=True)
        display(Image('model.png'))
        plt.show()
        model.summary()

    plt.rcParams["figure.figsize"] = (15, 8)

    if measure_performance:
        plt.plot(fitted_model.history['accuracy'])
        plt.plot(fitted_model.history['val_accuracy'])
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Test'], loc='upper left')
        plt.show()

        plt.plot(fitted_model.history['loss'])
        plt.plot(fitted_model.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Test'], loc='upper left')
        plt.show()

        y_pred = model.predict(test_batches)

        labels = [f'{ar} ({en})' for ar, en in zip(CLASSES_AR, CLASSES)]
        labels = [get_display(arabic_reshaper.reshape(label)) for label in labels]

        ax = sns.heatmap(confusion_matrix(test_batches.classes, y_pred.argmax(axis=1)), annot=True, cmap='Blues',
                         fmt='g')
        ax.set_title('Confusion Matrix')
        ax.set_xlabel('Predicted Values')
        ax.set_ylabel('Actual Values')
        ax.xaxis.set_ticklabels(labels)
        ax.yaxis.set_ticklabels(labels)
        plt.xticks(rotation=90)
        plt.yticks(rotation=0)
        plt.show()

    return fitted_model

# Capturing input

In [ ]:
# @title Seed Value { run: "auto", display-mode: "form" }

SEED = 42 # @param {type:"integer"}

print(Fore.GREEN + u'\u2713 ' + 'Attributes have been recorded successfully.\n')    
print(Style.RESET_ALL)

In [ ]:
# @title Neural Network Schematics { run: "auto", display-mode: "form" }

NEURAL_NETWORK_TYPE = NeuralNetworkType.CONVOLUTIONAL_NEURAL_NETWORK_WITH_TRANSFER_LEARNING  # @param ['NeuralNetworkType.FULLY_CONNECTED_NETWORK', 'NeuralNetworkType.CONVOLUTIONAL_NEURAL_NETWORK', 'NeuralNetworkType.CONVOLUTIONAL_NEURAL_NETWORK_WITH_TRANSFER_LEARNING'] {type:'raw'}
NEURAL_NETWORK_SIZE = NeuralNetworkSize.SMALL  # @param ['NeuralNetworkSize.NANO', 'NeuralNetworkSize.MICRO', 'NeuralNetworkSize.SMALL'] {type:'raw'}

print(Fore.GREEN + u'\u2713 ' + 'Attributes have been recorded successfully.\n')    
print(Style.RESET_ALL)

if 'INEFFECTIVE' in NEURAL_NETWORK_TYPE.value:
    print(Back.YELLOW + Fore.BLACK + u'\u26A0 ' +
          'An ineffective neural network model has been selected, this architecture may not be suitable for image tasks.')

if 'SCRATCH' in NEURAL_NETWORK_TYPE.value:
    print(Back.YELLOW + Fore.BLACK + u'\u26A0 ' +
          'A from-scratch neural network type has been selected, expected very long time runtimes to generalize.')
    
if 'INEFFECTIVE' in NEURAL_NETWORK_SIZE.value and NEURAL_NETWORK_TYPE != NeuralNetworkType.CONVOLUTIONAL_NEURAL_NETWORK_WITH_TRANSFER_LEARNING:
    print(Back.YELLOW + Fore.BLACK + u'\u26A0 ' +
          'A very small neural network size has been selected, expect poor results.')

In [ ]:
# @title Hyperparameters { run: "auto", display-mode: "form" }

HIDDEN_LAYER_ACTIVATION_FUNCTION_TYPE = HiddenLayerActivationFunctionType.RELU  # @param ['HiddenLayerActivationFunctionType.SIGMOID', 'HiddenLayerActivationFunctionType.TANH', 'HiddenLayerActivationFunctionType.RELU', 'HiddenLayerActivationFunctionType.LEAKY_RELU'] {type:'raw'}
OPTIMIZER_TYPE = OptimizerType.ADAM  # @param ['OptimizerType.SGD', 'OptimizerType.RMS_PROP', 'OptimizerType.ADAM'] {type:'raw'}
EPOCHS = 30  # @param {type:"slider", min:1, max:100, step:1}
EARLY_STOPPING_PATIENCE = 10  # @param {type:"integer"}

print(Fore.GREEN + u'\u2713 ' + 'Attributes have been recorded successfully.\n')
print(Style.RESET_ALL)

if 'INTENSIVE' in HIDDEN_LAYER_ACTIVATION_FUNCTION_TYPE.value:
    print(Back.YELLOW + Fore.BLACK + u'\u26A0 ' +
          'A computationally intensive activation function has been selected, expect long runtimes.')

if EPOCHS < 10:
    print(Back.YELLOW + Fore.BLACK + u'\u26A0 ' +
          'The number of epochs is low, the model may not generalize well.')

if EARLY_STOPPING_PATIENCE > 10:
    print(Back.YELLOW + Fore.BLACK + u'\u26A0 ' +
          'Patience value is high, expect long runtimes.')

print(Style.RESET_ALL)

# Methods Invocation

In [ ]:
finalize_seed()

In [ ]:
prime_dataset()

In [ ]:
model = build_model(measure_performance=True)